In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install openai

import os
import json
import openai
from openai import OpenAI
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import random

In [10]:

def get_description(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
              data = json.load(file)
        except Exception as e:
            print(f"无法读取文件 {file_path}: {e}")

        description=''
        for i in data['conversations']:
            conv_nospace=i['value'].replace(' ','')
            if conv_nospace.startswith("Intheframe") or conv_nospace.startswith("Inframe"):
                description=description+i['value'].replace('Identify the corresponding frame in the image based on this description.','')
        return description


def get_image_path(file_path):
  try:
            with open(file_path, 'r', encoding='utf-8') as file:
              data = json.load(file)
  except Exception as e:
            print(f"无法读取文件 {file_path}: {e}")

  image_name=data['image'].split('/')[-1]
  image_path='/content/drive/MyDrive/ft_data_v3_sjc_copy/images'+'/'+image_name
  return image_path



api_key = "sk-ba1d6ce7ae594afcb423e914765d3bd1"
base_url = 	"https://api.deepseek.com"
def get_response(message):

    client = OpenAI(api_key=api_key, base_url=base_url)

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant.You will read a description of a basketball tactic and then answer a question based on the description."},
            {"role": "user", "content": message},
        ],
        stream=False
    )

    return response.choices[0].message.content

In [ ]:
# 设置文件夹路径
directory = '/content/drive/MyDrive/ft_data_v3_sjc_copy/jsons'

action_Dict={
    'pick&roll':['pick','pnr'],
    'off ball screen':['screen','stagger','zipper','pin','elevator','flare','doubledrag'],
    'hands off':['handsoff','handoff','dho']
}

file_output='/content/drive/MyDrive/ft_data_v3_sjc_copy/question3_list.json'


for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            description=get_description(file_path)
            desc_normal=description.replace(' ','').replace('-','').replace('_','').lower()
            action_type=''

            if any(act in desc_normal for act in action_Dict['pick&roll']):
              action_type='pick&roll'
            elif any(act in desc_normal for act in action_Dict['off ball screen']):
              action_type='off ball screen'
            elif any(act in desc_normal for act in action_Dict['hands off']):
              action_type='hands off'
            else:
              print('No action detected!!!','\n')
              #print(desc_normal)

            if action_type :
              Sample_Dict={}
              Sample_Dict['id']=file_path
              Sample_Dict['image']=get_image_path(file_path)
              Sample_Dict['conversations']=[]

              cvst_human={}
              cvst_human['from']='human'
              question=f'Here is the description: {description}. Here is the question: These actions are known to be one {action_type}:{action_Dict[action_type]}, then which players do the {action_type} actions in this tatic ? Answer only with numbers separated by commas.'

              cvst_gpt={}
              cvst_gpt['from']='gpt'
              response=get_response(question)

              res_norm=str(response.replace(',','').replace(' ',''))
              player_list=list(set([i for i in res_norm]))

              player_q=random.choice(player_list)


              player_all=['1','2','3','4','5']

              try:
                  player_all.remove(player_q)
                  # 这里放置可能会引发异常的代码
                  options = {chr(65 + i): num for i, num in enumerate(player_all)}
                  options_str=''
                  for option, value in options.items():
                    options_str=options_str+f'{option}.{value}  '

                  cvst_human['value']=f'Here is the description: {description}. Here is the question: These actions are known to be one {action_type}:{action_Dict[action_type]}, then which player(s) do the {action_type} action with player{player_q} in this tatic ? {options_str}'

                  answer_selections=[key for key,value in options.items() if value in player_list]
                  answer_str=''
                  for s in answer_selections:
                    answer_str=answer_str+s



                  cvst_gpt['value']=answer_str

                  Sample_Dict['conversations'].append(cvst_human)
                  Sample_Dict['conversations'].append(cvst_gpt)

                  #print(Sample_Dict)


                  with open(file_output, 'r') as file:
                      data_list = json.load(file)
                  data_list.append(Sample_Dict)

                  with open(file_output, 'w') as file:
                    json.dump(data_list, file, indent=4)


              except Exception as e:
                  print(f"发生错误: {e}")
                  print(player_list)
                  print(f"变量 player_q的值是: {player_q}")
              #player_all.remove(player_q)

  0%|          | 3/6977 [00:05<3:08:15,  1.62s/it]

No action detected!!! 



  0%|          | 25/6977 [00:39<3:11:44,  1.65s/it]

No action detected!!! 



  0%|          | 28/6977 [00:42<2:34:23,  1.33s/it]

...

 98%|█████████▊| 6857/6977 [3:17:11<03:54,  1.95s/it]

No action detected!!! 



 99%|█████████▉| 6930/6977 [3:19:45<01:35,  2.03s/it]

No action detected!!! 



100%|██████████| 6977/6977 [3:21:21<00:00,  1.73s/it]
